In [1]:
import numpy as np
from tqdm import tqdm
from constants import MAX_SEQ_LEN

with open('../hw_train.csv', 'rb') as f:
    ls = f.readlines()[1:]
# count vocabulary and max sequence length
lyrics_map = {}
bos_token = 's'.decode('utf8')
eos_token = 'e'.decode('utf8')
none_lyrics_words = [u'編詞', u'作曲', u'作詞', u'編曲', u'監製']
vocab = [bos_token, eos_token]
lens = []
c = 0 
with tqdm(total=len(ls)) as pbar:
    for l in ls:
        l = l.decode('utf8')
        no, l = l.strip().split(',')
        no = int(no)
        if no not in lyrics_map:
            lyrics_map[no] = []
        lyrics_map[no].append(l)
        pbar.update(1)
# split into Q,K pair
with tqdm(total=len(lyrics_map)) as pbar:
    for k,v in lyrics_map.items():
        line_num = len(v)
        V = [bos_token,]
        Q = [bos_token,]
        for i,l in enumerate(v):
            exist_none_layrics_word = False
            for nw in none_lyrics_words:
                if nw in l:
                    exist_none_layrics_word = True
                    break
            if exist_none_layrics_word:
                continue
            for ws in l.strip().split(' '):
                for w in ws:
                    if i <= line_num//2:
                        V.append(w)
                    else:
                        Q.append(w)
                    vocab.append(w)
        Q.append(eos_token)
        V.append(eos_token)
        lens.append(len(Q))
        lens.append(len(V))
        lyrics_map[k] = (Q, V)
        pbar.update(1)
            
print 'vocabulary size : %d, max len : %.0f, mean : %.0f, std : %.0f' % (len(set(vocab)), np.max(lens), np.mean(lens), np.std(lens))
print 'lyrics number : %d' % len(lyrics_map)
# print np.max(lens), np.mean(lens), np.std(lens)



100%|██████████| 18338/18338 [00:02<00:00, 7678.56it/s]


vocabulary size : 6455, max len : 3575, mean : 161, std : 106
lyrics number : 18338


In [2]:
import numpy as np
import keras
from constants import MAX_SEQ_LEN, VOCAB_DIM
from keras.preprocessing.sequence import pad_sequences

texts = vocab

tokenizer = keras.preprocessing.text.Tokenizer(num_words=VOCAB_DIM, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ',
                                   lower=False, split=' ', char_level=True, oov_token=None)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Using TensorFlow backend.


Found 6455 unique tokens.


In [4]:
from constants import MAX_SEQ_LEN
from keras.preprocessing.sequence import pad_sequences

idx = np.random.permutation(len(lyrics_map))
val_num = len(lyrics_map)//10
train_idx, val_idx = idx[val_num:], idx[:val_num]
train_u_map = {k:lyrics_map[k] for k in lyrics_map.keys()[val_num:]}
val_u_map = {k:lyrics_map[k] for k in lyrics_map.keys()[:val_num]}
def batch_boostrap_generator(batch_size, lyrics_map, max_seq_len, tokenizer):
    G = boostrap_generator(lyrics_map, max_seq_len, tokenizer)
    while True:
        Q = []
        K = []
        K_len = []
        for i in range(batch_size):
            q, k, k_len = next(G)
            Q.append(q)
            K_len.append(k_len)
            K.append(k)
            
        yield np.vstack(Q), np.vstack(K), np.vstack(K_len)
def boostrap_generator(lyrics_map, max_seq_len, tokenizer):
    while True:
        keys = lyrics_map.keys()
        for idx in np.random.permutation(len(lyrics_map)):
            no = keys[idx]
            Q, K = lyrics_map[no]
            
            Q = tokenizer.texts_to_sequences([Q, ])
            Q = pad_sequences(Q, maxlen=max_seq_len, padding='post', truncating='post')
            K = tokenizer.texts_to_sequences([K, ])
            K_len = len(K)
            K = pad_sequences(K, maxlen=max_seq_len, padding='post', truncating='post')
            
            yield Q, K, K_len
    
bat = 4
G = batch_boostrap_generator(bat, train_u_map, MAX_SEQ_LEN, tokenizer)
val_G = batch_boostrap_generator(bat, val_u_map, MAX_SEQ_LEN, tokenizer)
q, k, k_len = next(G)
print q.shape, k.shape, k_len.shape


(4, 4500) (4, 4500) (4, 1)


### RNN

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import xavier_normal_
from constants import VOCAB_DIM, D_MODEL, MAX_SEQ_LEN
# construct neuron network
class Seq2seq(nn.Module):

    def __init__(self, dm, num_lay):
        super(Seq2seq, self).__init__()
#         for construct cache positional encoding matrix.
        self.emb = nn.Embedding(VOCAB_DIM+1, dm, padding_idx=0)
        self.encoder = nn.GRU(dm, dm, num_lay, batch_first=True)
        self.decoder = nn.GRU(dm, dm, num_lay, batch_first=True)
        self.linear = nn.Linear(dm, VOCAB_DIM+1)

    def forward(self, Q, K, K_len):
        K = self.emb(K)
        Q = self.emb(Q)
        en_out, _ = self.encoder(K) 
#         catch last valid hidden vector
        batch,_ = K_len.size()
        h_out = []
        for i in range(batch):
            k_len = K_len[i,0]
            h_out.append(en_out[i:i+1,k_len,:])
        h_out = torch.cat(h_out, dim=0).unsqueeze(0)
#         
        out,_  = self.decoder(Q, h_out)
        y = self.linear(out)
        return y

bat = 7
lay_num = 1
model = Seq2seq(D_MODEL, lay_num).cuda()
# print(o.size())
q, k, k_len = next(G)
q = torch.LongTensor(q).cuda()
k = torch.LongTensor(k).cuda()
k_len = torch.LongTensor(k_len).cuda()

q.requires_grad_(False)
k.requires_grad_(False)
k_len.requires_grad_(False)


output = model(q, k, k_len)
print(o.size())
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))



torch.Size([4, 4500, 6456])
1857336


In [ ]:
from collections import deque
from tqdm import tqdm as tqdm

import time
def dump_log(model, n_iter, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label, log_file_stream, tmp_model_path):
    log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f<split>%s<split>%s<split>%s<split>%s\n' % (n_iter, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label)
    log_file_stream.write(log_text.encode('utf8'))
    if n_iter % 10 == 0 :
        log_file_stream.flush()
        torch.save(model, tmp_model_path)
def normal_acc(pred, label):
    pred = torch.argmax(pred, dim=-1)
    mask = torch.ones_like(label, dtype=torch.uint8)
#     
    acc = pred == label
    acc = torch.sum(acc).item() / float(torch.sum(mask).item())
    
    return acc
def seq2text(output, index_word):
    assert len(output.size()) == 1
    seq = output
    s = ''
    for i in seq:
        i = int(i.item())
        if i == 0:
            continue
        w = index_word[i]
        s += w
    return s
    
def rev_mask(m):
    out = torch.ones_like(m, dtype=torch.uint8, requires_grad=False)
    out.masked_fill_(m, 0)
    return out
        
acc_q = deque(maxlen=100)
loss_q = deque(maxlen=10)

val_acc_q = deque(maxlen=100)
val_loss_q = deque(maxlen=10)

t = time.time()
best_loss = float('inf')

epochs = 100
batch_size = 4
G = batch_boostrap_generator(batch_size, train_u_map, MAX_SEQ_LEN, tokenizer)
val_G = batch_boostrap_generator(batch_size, val_u_map, MAX_SEQ_LEN, tokenizer)
# criterion = nn.CrossEntropyLoss(reduction='none')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
print 'start training.'
with open('log.txt', 'w') as f:
    with open('best.txt', 'w') as best_log:
        iters = 100000000
        with tqdm(total=iters) as pbar:
            for it in range(iters):
                optimizer.zero_grad()
                model.train()
                q, k, k_len = next(G)
                q = torch.LongTensor(q).cuda()
                k = torch.LongTensor(k).cuda()
                k_len = torch.LongTensor(k_len).cuda()

                q.requires_grad_(False)
                k.requires_grad_(False)
                k_len.requires_grad_(False)


                output = model(q, k, k_len)
                y = k[:,1:]
#                 loss = torch.sum(criterion(output, y) * pad_mask[:,1:, :]) 
                loss = criterion(output[:,:-1,:,].permute(0,2,1), y) 
                label = y

                pred = output[:,:-1,:,]
                acc = normal_acc(pred, label)
                acc_q.append(acc)
                train_pred = seq2text(torch.argmax(output[0,:,:], dim=-1), tokenizer.index_word)
                train_label= seq2text(k[0,:], tokenizer.index_word)
                loss.backward()
                loss_q.append(loss.item())
                
                optimizer.step()
                with torch.no_grad():
                    model.eval()
                    q, k, k_len = next(val_G)
                    q = torch.LongTensor(q).cuda()
                    k = torch.LongTensor(k).cuda()
                    k_len = torch.LongTensor(k_len).cuda()
                    
                    q.requires_grad_(False)
                    k.requires_grad_(False)
                    k_len.requires_grad_(False)
                    
                    output = model(q, k, k_len)
                    y = k[:,1:]
                    val_loss = criterion(output[:,:-1,:,].permute(0,2,1), y) 
                    label = y

                    pred = output[:,:-1,:,]
                    val_acc = normal_acc(pred, label)
                    val_acc_q.append(val_acc)
                    val_pred = seq2text(torch.argmax(output[0,:,:], dim=-1), tokenizer.index_word)
                    val_label= seq2text(k[0,:], tokenizer.index_word)
                    val_loss_q.append(val_loss.item())


                
                acc = np.mean(acc_q)
                val_acc = np.mean(val_acc_q)
                loss = np.mean(acc_q)
                val_loss = np.mean(val_acc_q)
                
#                     pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, loss : %.3f, val_loss : %.3f \t %.3f, %.3f, %.3f, %.3f' % (acc, val_acc, loss.item(), val_loss.item(), a,b,c,d), refresh=False)
                pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, loss : %.3f, val_loss : %.3f' % (acc, val_acc, loss.item(), val_loss.item()), refresh=False)
                pbar.update(batch_size)
                dump_log(model, (it+1)*batch_size, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label, f,'./tmp.pt')
                if val_loss.item() < best_loss and it > 100:
                    torch.save(model, './best.pt')
                    best_loss = val_loss
                    best_log.write('%d\t%.5f\n' % ((it+1)*batch_size, best_loss))
                    best_log.flush()
                if it % 100 == 0 and it >= 100: 
                    print 'train pred : %s\ntrain label : %s' % (train_pred, train_label)
                    print 'validation pred : %s\nvalidation label : %s' % (val_pred, val_label)

# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

  0%|          | 0/100000000 [00:00<?, ?it/s]

start training.


  0%|          | 20/100000000 [00:03<5302:07:41,  5.24it/s, acc : 0.000, val_acc : 0.194, loss : 0.000, val_loss : 0.194]/home/k123/env/python2.7.15/lib/python2.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Seq2seq. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
  0%|          | 404/100000000 [00:58<3910:12:43,  7.10it/s, acc : 0.922, val_acc : 0.936, loss : 0.922, val_loss : 0.936]

train pred : 你你你你你你你你你你
train label : s你会是我的自从见了你以后我就有预感我会牵你的手在不久之后我就要很温柔就要非常温柔我将你爱个够这是最后的决定了我已经决定为爱往前走在你蓝蓝的天空中我想要和你到云端吹风这是最后的决定了我已经决定爱你到最后在你小小的掌心中我想要送你美丽的彩虹你会是我的e
validation pred : 我你
validation label : s像是最初像是结果分不明谁去干的错我心跌堕啊罪是我闯罪是我当只要求由我去补错却不退让啊既做错血再流我也愿经过我已做错那怕由我去受伤过命运再阻莫用去躲e


  0%|          | 804/100000000 [01:54<3926:54:31,  7.07it/s, acc : 0.962, val_acc : 0.962, loss : 0.962, val_loss : 0.962]

...

  0%|          | 40004/100000000 [1:57:19<3942:49:31,  7.04it/s, acc : 0.964, val_acc : 0.966, loss : 0.964, val_loss : 0.966]

train pred : 看看看看看看看看看看看的你你你的的的的人的的轻的的你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你
train label : s看着你想着伊茫茫亲像着酒醉想起故乡心爱你不觉心内真稀微不是我无勇气不是我无爱你不愿给人看无起才着离别走千里吉他声唱情诗轻轻叫着你名字希望永远爱着你看着你想着伊茫茫亲像着酒醉想起故乡心爱你不觉心内真稀微不是我无勇气不是我无爱你e
validation pred : 词词吧過吧過大大大大大大大我我我我我我我我我我我我我我我我我我我的的的的的的的我我我我我我我我我我我我的的的的的的的的的的的的的的的的的的的的的的的的的的的一
validation label : s喇嘛轉世電吉他顧愷電吉他張立貝斯鄭芊蕙鼓黃河主唱沈雪晨和聲顧愷混音沈雪晨封面攝影黃河出品向日葵樂隊若干年後人們發現他已經轉世在西班牙他們到處找我從喀什米爾出發沿著恆河東南走在烈日下風雨中穿過鄉野村莊出入河曲和山啊他們分取两个方向在到处搜索忍耐著飢餓疲勞和錯誤他們進入古老的加利利準備順路探訪幼年耶穌的故居e


  0%|          | 40404/100000000 [1:58:16<3945:19:35,  7.04it/s, acc : 0.967, val_acc : 0.964, loss : 0.967, val_loss : 0.964]

train pred : 今曲情情情情情我情情不一一一一不不一一一心心心心心心心心心心心心心心心心心心心心心心心心心心我我我我我我不不我的的的的的的的的的的的的的的的的的的的的的的的的的的
train label : s愛太深容易看見傷痕情太真所以難捨難分折一千對紙鶴結一千顆心情傳說中心與心能相逢夜難眠往事乎隱乎現心在痛對你越陷越深折一千對紙鶴解一千顆心願夢醒後情緣不再飄零我的心不後悔折折疊疊都是為了你我的淚流不盡糾纏在夢裡夜裡的負累我的心不後悔反反覆覆也是為了你千紙鶴千顆心在風裡飛愛太深容易看見傷痕e
validation pred : 没没没没没没没没没没没没没没没有过过过过过过过过过一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一一不不不不不不不一不一一一一一一一一一一不不不不不不不不不不一一一一一
validation label : s没有什么不能散没有什么不能断没有什么不能承担没有什么不能藏没有什么不能放没有什么不能遗忘谁在我青春最结尾烙下伤疤未来一天一天一天枯萎腐化以为爱了就能别无所求以为快乐就是全部全部拥有等到热情热切热烈冷却成无动于衷怀念一往无前的懵懂没有什么能永久没有什么不能走没有什么不能破e


  0%|          | 40420/100000000 [1:58:18<3953:10:07,  7.02it/s, acc : 0.967, val_acc : 0.964, loss : 0.967, val_loss : 0.964]